In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.2
  UserWarning)


In [3]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [4]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit50.csv')

In [5]:
project_releases = project_releases[project_releases['project_name'] == 'Okhttp'].head(4)

In [6]:
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
897,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",2021-01-30 19:44:19
898,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",2021-01-30 18:36:37
899,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",2020-10-07 03:28:26
900,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",2021-01-30 18:09:57


In [7]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results/Okhttp.csv')
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched,latest_commit_date,commit_compared_with_date,author,author_email
0,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,1,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
1,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.RecordedRequestTest,class,7,5,2,6,0,5,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,9.0,90,2,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
2,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.internal.http2.Http2Server,class,9,33,2,53,32,9,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,86.0,147,3,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
3,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.MockWebServerTest,class,22,54,1,111,0,40,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,321.0,566,4,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
4,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,5,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca


In [8]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [9]:
print(unique_commit)
print(unique_releases_commit)

['3e331c108905a97fa9718b40844ddc1356fc86b5'
 'f8065acac28a0b86f3bdbd854ac87af0789416d0'
 '963c76864344d0d55ba7421f474c130a958a9649'
 '59ae0c138d6468aac5d92781b19cddc26e04e862'
 '4677beea96f0afb1e061f119e52e0203d4cd3738'
 '4036fa55b72faaad3a14f806094184ade157917d'
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8'
 '083315a473d8b2fc972a70449272b296683ac38c'
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1'
 '334eb0072512155e57a809b2fa18861c0d68ec9f']
['b84627ef4bc43f2096c1b764720f9ff2387dca32'
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1'
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652']


In [10]:
list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

['4fd1e8f99833eebdd2e99f3456322aa197f5e652',
 '4036fa55b72faaad3a14f806094184ade157917d',
 '59ae0c138d6468aac5d92781b19cddc26e04e862',
 'f8065acac28a0b86f3bdbd854ac87af0789416d0',
 'b84627ef4bc43f2096c1b764720f9ff2387dca32',
 '3e331c108905a97fa9718b40844ddc1356fc86b5',
 '083315a473d8b2fc972a70449272b296683ac38c',
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1',
 '334eb0072512155e57a809b2fa18861c0d68ec9f',
 '963c76864344d0d55ba7421f474c130a958a9649',
 '4677beea96f0afb1e061f119e52e0203d4cd3738',
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8',
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1']

In [10]:
project_link = "https://github.com/square/okhttp"

In [18]:

owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/{owner}/{repo}/contributors"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/repos/square/okhttp/contributors
[{'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
  'contributions': 2592,
  'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
  'followers_url': 'https://api.github.com/users/swankjesse/followers',
  'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
  'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
  'gravatar_id': '',
  'html_url': 'https://github.com/swankjesse',
  'id': 133019,
  'login': 'swankjesse',
  'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
  'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
  'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
  'repos_url': 'https://api.github.com/users/swankjesse/repos',
  'site_admin': False,
  'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/swankjesse/subscri

In [20]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/users/swankjesse
{'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
 'bio': None,
 'blog': 'http://publicobject.com/',
 'company': 'Square, Inc.',
 'created_at': '2009-09-30T02:51:25Z',
 'email': 'jwilson@squareup.com',
 'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
 'followers': 5667,
 'followers_url': 'https://api.github.com/users/swankjesse/followers',
 'following': 0,
 'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
 'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': None,
 'html_url': 'https://github.com/swankjesse',
 'id': 133019,
 'location': 'Waterloo, Ontario, Canada',
 'login': 'swankjesse',
 'name': 'Jesse Wilson',
 'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
 'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
 'public_gists': 91,
 'public_repos': 22,
 'received_events_url': 'https://api.github.com/users/swankj

In [24]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
query_url = "https://api.github.com/users/swankjesse/repos"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

print(len(r.json()))

https://api.github.com/users/swankjesse/repos
[{'allow_forking': True,
  'archive_url': 'https://api.github.com/repos/swankjesse/compose-samples/{archive_format}{/ref}',
  'archived': False,
  'assignees_url': 'https://api.github.com/repos/swankjesse/compose-samples/assignees{/user}',
  'blobs_url': 'https://api.github.com/repos/swankjesse/compose-samples/git/blobs{/sha}',
  'branches_url': 'https://api.github.com/repos/swankjesse/compose-samples/branches{/branch}',
  'clone_url': 'https://github.com/swankjesse/compose-samples.git',
  'collaborators_url': 'https://api.github.com/repos/swankjesse/compose-samples/collaborators{/collaborator}',
  'comments_url': 'https://api.github.com/repos/swankjesse/compose-samples/comments{/number}',
  'commits_url': 'https://api.github.com/repos/swankjesse/compose-samples/commits{/sha}',
  'compare_url': 'https://api.github.com/repos/swankjesse/compose-samples/compare/{base}...{head}',
  'contents_url': 'https://api.github.com/repos/swankjesse/compos

  'open_issues': 0,
  'open_issues_count': 0,
  'owner': {'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
            'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
            'followers_url': 'https://api.github.com/users/swankjesse/followers',
            'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
            'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
            'gravatar_id': '',
            'html_url': 'https://github.com/swankjesse',
            'id': 133019,
            'login': 'swankjesse',
            'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
            'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
            'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
            'repos_url': 'https://api.github.com/users/swankjesse/repos',
            'site_admin': False,
            'starred_url': 'https://api.github.co

            'site_admin': False,
            'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
            'subscriptions_url': 'https://api.github.com/users/swankjesse/subscriptions',
            'type': 'User',
            'url': 'https://api.github.com/users/swankjesse'},
  'permissions': {'admin': False,
                  'maintain': False,
                  'pull': True,
                  'push': False,
                  'triage': False},
  'private': False,
  'pulls_url': 'https://api.github.com/repos/swankjesse/micronaut-core/pulls{/number}',
  'pushed_at': '2021-02-09T04:27:36Z',
  'releases_url': 'https://api.github.com/repos/swankjesse/micronaut-core/releases{/id}',
  'size': 69769,
  'ssh_url': 'git@github.com:swankjesse/micronaut-core.git',
  'stargazers_count': 0,
  'stargazers_url': 'https://api.github.com/repos/swankjesse/micronaut-core/stargazers',
  'statuses_url': 'https://api.github.com/repos/swankjesse/micronaut-core/statuses/{sha}',
 

In [23]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
query_url = 'https://api.github.com/repos/swankjesse/protobuf/languages'
query_url = 'https://api.github.com/repos/swankjesse/compose-samples/languages'
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/repos/swankjesse/compose-samples/languages
{'Kotlin': 157439, 'Shell': 1102}


In [ ]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

In [30]:

r = requests.get("https://api.github.com/search/code?q=addClass+user:mozilla",headers=headers)
pprint(r.json())

{'incomplete_results': False,
 'items': [{'git_url': 'https://api.github.com/repositories/10508666/git/blobs/32e18fbecdf1eee3dcda3e7877ea60bdaae4de06',
            'html_url': 'https://github.com/mozilla/picl-mocks/blob/7478d06de1977a1ad406135015d0bb7ee4f7b692/public/javascripts/one.js',
            'name': 'one.js',
            'path': 'public/javascripts/one.js',
            'repository': {'archive_url': 'https://api.github.com/repos/mozilla/picl-mocks/{archive_format}{/ref}',
                           'assignees_url': 'https://api.github.com/repos/mozilla/picl-mocks/assignees{/user}',
                           'blobs_url': 'https://api.github.com/repos/mozilla/picl-mocks/git/blobs{/sha}',
                           'branches_url': 'https://api.github.com/repos/mozilla/picl-mocks/branches{/branch}',
                           'collaborators_url': 'https://api.github.com/repos/mozilla/picl-mocks/collaborators{/collaborator}',
                           'comments_url': 'https://api.g

                                     'subscriptions_url': 'https://api.github.com/users/mozilla/subscriptions',
                                     'type': 'Organization',
                                     'url': 'https://api.github.com/users/mozilla'},
                           'private': False,
                           'pulls_url': 'https://api.github.com/repos/mozilla/remo/pulls{/number}',
                           'releases_url': 'https://api.github.com/repos/mozilla/remo/releases{/id}',
                           'stargazers_url': 'https://api.github.com/repos/mozilla/remo/stargazers',
                           'statuses_url': 'https://api.github.com/repos/mozilla/remo/statuses/{sha}',
                           'subscribers_url': 'https://api.github.com/repos/mozilla/remo/subscribers',
                           'subscription_url': 'https://api.github.com/repos/mozilla/remo/subscription',
                           'tags_url': 'https://api.github.com/repos/mozilla/remo/t

                           'url': 'https://api.github.com/repos/mozilla/popcorn-editor'},
            'score': 1.0,
            'sha': '7130282b08532c112e461d6abda8b18064544c98',
            'url': 'https://api.github.com/repositories/36961669/contents/PopcornEditor/static/bower/bootstrap/js/collapse.js?ref=901c5c65faee66b024a89f5f6324ae20c653bafb'},
           {'git_url': 'https://api.github.com/repositories/8780590/git/blobs/18d8009f0f2a44dbac08f2b6f7af23fe6b5624d5',
            'html_url': 'https://github.com/mozilla/webmaker.org/blob/dc87802c9d4122550768c2adfad0c31fcab2f717/public/js/base/login.js',
            'name': 'login.js',
            'path': 'public/js/base/login.js',
            'repository': {'archive_url': 'https://api.github.com/repos/mozilla/webmaker.org/{archive_format}{/ref}',
                           'assignees_url': 'https://api.github.com/repos/mozilla/webmaker.org/assignees{/user}',
                           'blobs_url': 'https://api.github.com/repos/mozilla/

In [19]:
r.links

{'next': {'url': 'https://api.github.com/repositories/5152285/commits?page=31&per_page=100',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/5152285/commits?page=50&per_page=100',
  'rel': 'last'},
 'first': {'url': 'https://api.github.com/repositories/5152285/commits?page=1&per_page=100',
  'rel': 'first'},
 'prev': {'url': 'https://api.github.com/repositories/5152285/commits?page=29&per_page=100',
  'rel': 'prev'}}

In [18]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/commits?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/repos/square/okhttp/commits?page=30&per_page=100
[{'author': {'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
             'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
             'followers_url': 'https://api.github.com/users/swankjesse/followers',
             'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
             'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
             'gravatar_id': '',
             'html_url': 'https://github.com/swankjesse',
             'id': 133019,
             'login': 'swankjesse',
             'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
             'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
             'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
             'repos_url': 'https://api.github.com/users/swankjesse/repos',
             'site_admin': False,
         

             'type': 'User',
             'url': 'https://api.github.com/users/iagreen'},
  'comments_url': 'https://api.github.com/repos/square/okhttp/commits/597c279dfe51daf30457da85cc033feb34325286/comments',
  'commit': {'author': {'date': '2015-09-09T12:47:09Z',
                        'email': 'isaac@wink.com',
                        'name': 'Isaac Green'},
             'comment_count': 0,
             'committer': {'date': '2015-09-09T13:07:00Z',
                           'email': 'isaac@wink.com',
                           'name': 'Isaac Green'},
             'message': 'only encode URI problem chars when actually '
                        'converting to URI',
             'tree': {'sha': '4f158072c7d888de598722987356d8f664431cf9',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/4f158072c7d888de598722987356d8f664431cf9'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/597c279dfe51daf30457da85cc033feb34325286',
 

             'repos_url': 'https://api.github.com/users/JakeWharton/repos',
             'site_admin': False,
             'starred_url': 'https://api.github.com/users/JakeWharton/starred{/owner}{/repo}',
             'subscriptions_url': 'https://api.github.com/users/JakeWharton/subscriptions',
             'type': 'User',
             'url': 'https://api.github.com/users/JakeWharton'},
  'comments_url': 'https://api.github.com/repos/square/okhttp/commits/22c6eb7ecb4a3e6115f70f5f3c1c5c4e0d49b9ab/comments',
  'commit': {'author': {'date': '2015-08-26T01:21:40Z',
                        'email': 'jakewharton@gmail.com',
                        'name': 'Jake Wharton'},
             'comment_count': 0,
             'committer': {'date': '2015-08-26T01:21:40Z',
                           'email': 'jakewharton@gmail.com',
                           'name': 'Jake Wharton'},
             'message': '2.5.0 is latest.',
             'tree': {'sha': '4ac8683e7fd701a3fb9fd9cac1abcf6d2188faba',
  

  'commit': {'author': {'date': '2015-08-03T05:32:13Z',
                        'email': 'jesse@swank.ca',
                        'name': 'Jesse Wilson'},
             'comment_count': 0,
             'committer': {'date': '2015-08-03T05:32:13Z',
                           'email': 'jesse@swank.ca',
                           'name': 'Jesse Wilson'},
             'message': 'Merge pull request #1786 from '
                        'square/jwilson_0803_default_user_agent\n'
                        '\n'
                        'Make sure the default user agent is ASCII.',
             'tree': {'sha': 'c2f50df63779ec60daa01682e176274ed7b51502',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/c2f50df63779ec60daa01682e176274ed7b51502'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/e19b7913eb0ec73a24872c0abce85896728a9648',
             'verification': {'payload': None,
                              'reason': 'unsigned',
     

                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/5576502b317b28618ce79f41f958364f2e737661'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/8f5e344aba7b734efa61977651838a88d288a187',
             'verification': {'payload': None,
                              'reason': 'unsigned',
                              'signature': None,
                              'verified': False}},
  'committer': {'avatar_url': 'https://avatars.githubusercontent.com/u/66577?v=4',
                'events_url': 'https://api.github.com/users/JakeWharton/events{/privacy}',
                'followers_url': 'https://api.github.com/users/JakeWharton/followers',
                'following_url': 'https://api.github.com/users/JakeWharton/following{/other_user}',
                'gists_url': 'https://api.github.com/users/JakeWharton/gists{/gist_id}',
                'gravatar_id': '',
                'html_url': 'https://github.com/JakeWharton',
        

             'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
             'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
             'gravatar_id': '',
             'html_url': 'https://github.com/swankjesse',
             'id': 133019,
             'login': 'swankjesse',
             'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
             'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
             'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
             'repos_url': 'https://api.github.com/users/swankjesse/repos',
             'site_admin': False,
             'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
             'subscriptions_url': 'https://api.github.com/users/swankjesse/subscriptions',
             'type': 'User',
             'url': 'https://api.github.com/users/swankjesse'},
  'comments_url': 'https://api.github.com/repos/square

                           'name': 'jwilson'},
             'message': 'Modify timeout exception / update test expectations\n'
                        '\n'
                        'Related to issue #1676.\n'
                        'Depends on https://github.com/square/okio/pull/154\n'
                        '\n'
                        'URLConnection should throw SocketTimeoutException for '
                        'timeouts\n'
                        'not InterruptedIOException.',
             'tree': {'sha': '33d682684b74733aab643618000e04381bb3ac09',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/33d682684b74733aab643618000e04381bb3ac09'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/5af1d7f449509597f16bebbf0ef6c68d7c924398',
             'verification': {'payload': None,
                              'reason': 'unsigned',
                              'signature': None,
                              'verified': Fa

In [15]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100
[{'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'NONE',
  'body': '**Problem**\r\n'
          '\r\n'
          'Pending messages queue is capped, and connection gets closed if new '
          'message hits queue limit. \r\n'
          'I have 2 issues there:\r\n'
          '\r\n'
          '* Its hard to efficiently send big payloads (e.g. big file sliced '
          'into chunks) without resorting to polling `WebSocket.queueSize`\r\n'
          '* If new message exceeds queue size (16 MiB currently, not exposed '
          '), I would like to signal that to client  without closing '
          'connection.\r\n'
          '\r\n'
          '**Possible solution**\r\n'
          '  \r\n'
          '* Enhance `WebsocketListener` with `onMessageWritten(WebSocket)`\r\n'
          '* Enhance `Websocket` with `maxQueueSize` (it has `queueSize` '
          'already)',
  'closed_at'

           'gists_url': 'https://api.github.com/users/Aks-4125/gists{/gist_id}',
           'gravatar_id': '',
           'html_url': 'https://github.com/Aks-4125',
           'id': 29367708,
           'login': 'Aks-4125',
           'node_id': 'MDQ6VXNlcjI5MzY3NzA4',
           'organizations_url': 'https://api.github.com/users/Aks-4125/orgs',
           'received_events_url': 'https://api.github.com/users/Aks-4125/received_events',
           'repos_url': 'https://api.github.com/users/Aks-4125/repos',
           'site_admin': False,
           'starred_url': 'https://api.github.com/users/Aks-4125/starred{/owner}{/repo}',
           'subscriptions_url': 'https://api.github.com/users/Aks-4125/subscriptions',
           'type': 'User',
           'url': 'https://api.github.com/users/Aks-4125'}},
 {'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'NONE',
  'body': 'When using http-proxy, and the request belongs to https, it can not '
          '

          '[WARNING] public List<InetAddress> allocate(int count) {\r\n'
          '[WARNING] ^\r\n'
          '[WARNING] Javadoc Warnings\r\n'
          '[WARNING] '
          '/Users/yschimke/workspace/okhttp/samples/slack/src/main/java/okhttp3/slack/SlackApi.java:61: '
          'warning: no @param for scopes\r\n'
          '[WARNING] public HttpUrl authorizeUrl(String scopes, HttpUrl '
          'redirectUrl, ByteString state, String team) {\r\n'
          '[WARNING] ^\r\n'
          '[WARNING] '
          '/Users/yschimke/workspace/okhttp/samples/slack/src/main/java/okhttp3/slack/SlackApi.java:61: '
          'warning: no @param for redirectUrl\r\n'
          '[WARNING] public HttpUrl authorizeUrl(String scopes, HttpUrl '
          'redirectUrl, ByteString state, String team) {\r\n'
          '[WARNING] ^\r\n'
          '[WARNING] '
          '/Users/yschimke/workspace/okhttp/samples/slack/src/main/java/okhttp3/slack/SlackApi.java:61: '
          'warning: no @param for state\r\n'

          '[INFO] --- exec-maven-plugin:1.6.0:exec (default-cli) @ benchmarks '
          '---\r\n'
          'Experiment selection: \r\n'
          '  Instruments:   [arbitrary]\r\n'
          '  User parameters:   {bodyByteCount=[128, 1048576], chunked=[true, '
          'false], client=[OkHttp, OkHttpAsync, Apache, UrlConnection, Netty], '
          'concurrencyLevel=[1, 10], gzip=[true, false], headerCount=[0, 20], '
          'targetBacklog=[10], tls=[true, false]}\r\n'
          '  Virtual machines:  [default]\r\n'
          '  Selection type:    Full cartesian product\r\n'
          '\r\n'
          'This selection yields 320 experiments.\r\n'
          'com.google.caliper.runner.UserCodeException: An exception was '
          'thrown from the benchmark code\r\n'
          '\tat '
          'com.google.caliper.runner.ArbitraryMeasurementInstrument.dryRun(ArbitraryMeasurementInstrument.java:89)\r\n'
          '\tat '
          'com.google.caliper.runner.ExperimentingCaliperRun.dr

           'login': 'NightlyNexus',
           'node_id': 'MDQ6VXNlcjQwMzI2Njc=',
           'organizations_url': 'https://api.github.com/users/NightlyNexus/orgs',
           'received_events_url': 'https://api.github.com/users/NightlyNexus/received_events',
           'repos_url': 'https://api.github.com/users/NightlyNexus/repos',
           'site_admin': False,
           'starred_url': 'https://api.github.com/users/NightlyNexus/starred{/owner}{/repo}',
           'subscriptions_url': 'https://api.github.com/users/NightlyNexus/subscriptions',
           'type': 'User',
           'url': 'https://api.github.com/users/NightlyNexus'}},
 {'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'NONE',
  'body': 'The chain.connection() . in LoggingInterceptor from the example is '
          'always null.\r\n'
          'I am so confused.\r\n'
          'In the CallServerInterceptor, the chain.connection() is not '
          'null.But, after that,in Loggi

              'url': 'https://api.github.com/repos/square/okhttp/labels/needs%20info'},
             {'color': '0e8a16',
              'default': False,
              'description': 'Relates to usage specifically on Android',
              'id': 708275061,
              'name': 'android',
              'node_id': 'MDU6TGFiZWw3MDgyNzUwNjE=',
              'url': 'https://api.github.com/repos/square/okhttp/labels/android'}],
  'labels_url': 'https://api.github.com/repos/square/okhttp/issues/3784/labels{/name}',
  'locked': False,
  'milestone': {'closed_at': '2018-11-17T05:09:03Z',
                'closed_issues': 41,
                'created_at': '2018-11-03T23:09:25Z',
                'creator': {'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
                            'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
                            'followers_url': 'https://api.github.com/users/swankjesse/followers',
                          

           'events_url': 'https://api.github.com/users/liedQM/events{/privacy}',
           'followers_url': 'https://api.github.com/users/liedQM/followers',
           'following_url': 'https://api.github.com/users/liedQM/following{/other_user}',
           'gists_url': 'https://api.github.com/users/liedQM/gists{/gist_id}',
           'gravatar_id': '',
           'html_url': 'https://github.com/liedQM',
           'id': 33150549,
           'login': 'liedQM',
           'node_id': 'MDQ6VXNlcjMzMTUwNTQ5',
           'organizations_url': 'https://api.github.com/users/liedQM/orgs',
           'received_events_url': 'https://api.github.com/users/liedQM/received_events',
           'repos_url': 'https://api.github.com/users/liedQM/repos',
           'site_admin': False,
           'starred_url': 'https://api.github.com/users/liedQM/starred{/owner}{/repo}',
           'subscriptions_url': 'https://api.github.com/users/liedQM/subscriptions',
           'type': 'User',
           'url': 'htt

           'node_id': 'MDQ6VXNlcjg0OTA1ODU=',
           'organizations_url': 'https://api.github.com/users/zahidgirnar/orgs',
           'received_events_url': 'https://api.github.com/users/zahidgirnar/received_events',
           'repos_url': 'https://api.github.com/users/zahidgirnar/repos',
           'site_admin': False,
           'starred_url': 'https://api.github.com/users/zahidgirnar/starred{/owner}{/repo}',
           'subscriptions_url': 'https://api.github.com/users/zahidgirnar/subscriptions',
           'type': 'User',
           'url': 'https://api.github.com/users/zahidgirnar'}},
 {'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'NONE',
  'body': 'What kind of issue is this?\r\n'
          '\r\n'
          ' - [x] Bug report. If you’ve found a bug, spend the time to write a '
          'failing test. Bugs with tests\r\n'
          '       get fixed. Here’s an example: '
          'https://gist.github.com/swankjesse/981fcae102f513

In [14]:
r.json()[1]

{'sha': '8e996aadc5d7839f249399abac657e3c9df1dc4a',
 'node_id': 'MDY6Q29tbWl0NTE1MjI4NTo4ZTk5NmFhZGM1ZDc4MzlmMjQ5Mzk5YWJhYzY1N2UzYzlkZjFkYzRh',
 'commit': {'author': {'name': 'cketti',
   'email': 'cketti@gmail.com',
   'date': '2015-09-18T04:58:36Z'},
  'committer': {'name': 'cketti',
   'email': 'cketti@gmail.com',
   'date': '2015-09-18T04:58:36Z'},
  'message': "Fix bug where request body isn't sent when it should",
  'tree': {'sha': '00e460c581ba99883e6f50e3fb1b896f0f690304',
   'url': 'https://api.github.com/repos/square/okhttp/git/trees/00e460c581ba99883e6f50e3fb1b896f0f690304'},
  'url': 'https://api.github.com/repos/square/okhttp/git/commits/8e996aadc5d7839f249399abac657e3c9df1dc4a',
  'comment_count': 0,
  'verification': {'verified': False,
   'reason': 'unsigned',
   'signature': None,
   'payload': None}},
 'url': 'https://api.github.com/repos/square/okhttp/commits/8e996aadc5d7839f249399abac657e3c9df1dc4a',
 'html_url': 'https://github.com/square/okhttp/commit/8e996aadc5d7

In [19]:
import json
with open('raw_issues_github_json/Okhttp.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)

In [22]:
with open('raw_issues_github_json/Okhttp.json',) as f:
    data = json.load(f)

print(data)
print(data['resources'])

{'resources': {'core': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1636302250}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1636302250}, 'code_scanning_upload': {'limit': 500, 'used': 0, 'remaining': 500, 'reset': 1636305790}, 'actions_runner_registration': {'limit': 10000, 'used': 0, 'remaining': 10000, 'reset': 1636305790}}, 'rate': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}}
{'core': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1636302250}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636

In [20]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 4956, 'reset': 1636356890, 'used': 44},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636360178,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636360178,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 4956,
                        'reset': 1636356890,
                        'used': 44},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636360178,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
            